In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
import pandas as pd
import numpy as np

/Users/subir/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/subir/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/Users/subir/anaconda3/lib/python3.6/site-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.22) or chardet (2.3.0) doesn't match a supported version!
  RequestsDependencyWarning)


In [6]:
train=pd.read_csv('train_E6oV3lV.csv')
test=pd.read_csv('test_tweets_anuFYb8.csv')

In [7]:
train.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [8]:
test.head()

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedic...
1,31964,@user #white #supremacists want everyone to s...
2,31965,safe ways to heal your #acne!! #altwaystohe...
3,31966,is the hp and the cursed child book up for res...
4,31967,"3rd #bihday to my amazing, hilarious #nephew..."


In [9]:
target=train.label
train=train.drop('label',1)
data=train.append(test)

In [10]:
data['tweet']=data['tweet'].apply(lambda x: x.lower())
data['tweet']=data['tweet'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

In [11]:
data['tweet']=data['tweet'].apply(lambda x: x.replace('user',''))

In [12]:
data.head()

,id,tweet
0,1,when a father is dysfunctional and is so sel...
1,2,thanks for lyft credit i cant use cause they...
2,3,bihday your majesty
3,4,model i love u take with u all the time in u...
4,5,factsguide society now motivation


In [13]:
max_fatures = 5000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['tweet'].values)
X = tokenizer.texts_to_sequences(data['tweet'].values)
X = pad_sequences(X)

In [14]:
train_Seq=X[:len(train)]
test_Seq=X[len(train):]

In [15]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1], dropout=0.2))
model.add(LSTM(lstm_out, dropout_U=0.2, dropout_W=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

/Users/subir/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  """
/Users/subir/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(196, dropout=0.2, recurrent_dropout=0.2)`
  


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 34, 128)           640000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 394       
Total params: 895,194
Trainable params: 895,194
Non-trainable params: 0
_________________________________________________________________
None


In [16]:
Y = pd.get_dummies(target).values
X_train, X_test, Y_train, Y_test = train_test_split(train_Seq,Y, test_size = 0.3, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(22373, 34) (22373, 2)
(9589, 34) (9589, 2)


In [17]:
batch_size = 32
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=2, batch_size=32)

Train on 22373 samples, validate on 9589 samples
Epoch 1/2
22373/22373 [==============================] - 60s 3ms/step - loss: 0.1661 - acc: 0.9472 - val_loss: 0.1319 - val_acc: 0.9548
Epoch 2/2
22373/22373 [==============================] - 63s 3ms/step - loss: 0.0890 - acc: 0.9687 - val_loss: 0.1343 - val_acc: 0.9569


In [19]:
classes=model.predict_classes(test_Seq)

In [21]:
pred=model.predict(test_Seq)

In [24]:
pred

array([[9.71019924e-01, 2.89801341e-02],
       [3.62277269e-01, 6.37722671e-01],
       [9.99792993e-01, 2.06985351e-04],
       ...,
       [9.59704041e-01, 4.02959995e-02],
       [9.99888897e-01, 1.11103676e-04],
       [9.99534249e-01, 4.65791934e-04]], dtype=float32)

In [25]:
classes


array([0, 1, 0, ..., 0, 0, 0])